### Advance Regression Case Study on Housing Data Using Lasso and Ridge

A US-based housing company named Surprise Housing has decided to enter the Australian market. The company uses data analytics to purchase houses at a price below their actual values and flip them on at a higher price. For the same purpose, the company has collected a data set from the sale of houses in Australia. The data is provided in the CSV file below.

 

The company is looking at prospective properties to buy to enter the market. You are required to build a regression model using regularisation in order to predict the actual value of the prospective properties and decide whether to invest in them or not.

 

The company wants to know:

- Which variables are significant in predicting the price of a house, and

- How well those variables describe the price of a house.

### Data Understanding

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model, metrics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

import os

# hide warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# reading the dataset

housing= pd.read_csv('../input/house-prices-data/train.csv')

housing.head()

In [ ]:
# summary of the dataset:

print(housing.info())

In [ ]:
housing.describe()

#### Outlier Treatment
We will check for the outliers, if any then we will discard it.

In [ ]:
housing.describe(percentiles=[.25, .5, .75, .90, .95, .99])

#### We will manually pick a number of variables which appears to have high outliers

In [ ]:
#SalePrice

plt.boxplot(housing['SalePrice'])
Q1 = housing['SalePrice'].quantile(0.1)
Q3 = housing['SalePrice'].quantile(0.9)
IQR = Q3 - Q1
housing = housing[(housing['SalePrice'] >= Q1 - 1.5*IQR) & 
                      (housing['SalePrice'] <= Q3 + 1.5*IQR)]
housing.shape

In [ ]:
# Lot Area
plt.boxplot(housing['LotArea'])
Q1 = housing['LotArea'].quantile(0.1)
Q3 = housing['LotArea'].quantile(0.9)
IQR = Q3 - Q1
housing = housing[(housing['LotArea'] >= Q1 - 1.5*IQR) & 
                      (housing['LotArea'] <= Q3 + 1.5*IQR)]
housing.shape

In [ ]:
# MiscVal

plt.boxplot(housing['MiscVal'])
Q1 = housing['MiscVal'].quantile(0.1)
Q3 = housing['MiscVal'].quantile(0.9)
IQR = Q3 - Q1
housing = housing[(housing['MiscVal'] >= Q1 - 1.5*IQR) & 
                      (housing['MiscVal'] <= Q3 + 1.5*IQR)]
housing.shape

In [ ]:
# LotFrontage
plt.boxplot(housing['LotFrontage'])
Q1 = housing['LotFrontage'].quantile(0.1)
Q3 = housing['LotFrontage'].quantile(0.9)
IQR = Q3 - Q1
housing = housing[(housing['LotFrontage'] >= Q1 - 1.5*IQR) & 
                      (housing['LotFrontage'] <= Q3 + 1.5*IQR)]
housing.shape

In [ ]:
# MasVnrArea
plt.boxplot(housing['MasVnrArea'])
Q1 = housing['MasVnrArea'].quantile(0.1)
Q3 = housing['MasVnrArea'].quantile(0.9)
IQR = Q3 - Q1
housing = housing[(housing['MasVnrArea'] >= Q1 - 1.5*IQR) & 
                      (housing['MasVnrArea'] <= Q3 + 1.5*IQR)]
housing.shape

#### No we will look into the corelation matrix

In [ ]:
# correlation matrix
cor = housing.corr()
cor

### Data Cleaning

#### Firstly let check percentage of missing values

In [ ]:
# Checking the percentage of missing values
missing = round(100*(housing.isnull().sum()/len(housing.Id)), 2)
missing.loc[missing > 0]

#### Lets remove all the columns with more than 70% missing values. 

In [ ]:
columns_with_missing_values = list(missing[missing >= 70].index)

len(columns_with_missing_values)

#### There were **4 columns** with more than 70% values as missing. Removing such columns as these columns clearly add noise and wont help in analysis.

In [ ]:
housing = housing.drop(columns_with_missing_values,axis=1)
housing.shape

#### Treating Null Values

In [ ]:
#NA in FireplaceQu column means No Fireplace, so we will replace NA by it.
housing['FireplaceQu'].fillna('No Fireplace', inplace=True)

In [ ]:
housing['MasVnrArea'].fillna(0, inplace=True) 

In [ ]:
housing['LotFrontage'].fillna(0, inplace=True) 

In [ ]:
#NA in GarageType, GarageFinish, GarageQual, GarageCond columns mean No Garage, so we will replace NA by it.

housing['GarageType'].fillna('No Garage', inplace=True) 
housing['GarageFinish'].fillna('No Garage', inplace=True) 
housing['GarageQual'].fillna('No Garage', inplace=True) 
housing['GarageCond'].fillna('No Garage', inplace=True)

In [ ]:
#converting year to number of years
housing['YearBuilt'] = 2020 - housing['YearBuilt']
housing['YearRemodAdd'] = 2020 - housing['YearRemodAdd']
housing['GarageYrBlt'] = 2020 - housing['GarageYrBlt']
housing['YrSold'] = 2020 - housing['YrSold']

In [ ]:
#converting from int type to object to treat the variables as categorical variables

housing['MSSubClass'] = housing['MSSubClass'].astype('object')
housing['OverallQual'] = housing['OverallQual'].astype('object')
housing['OverallCond'] = housing['OverallCond'].astype('object')
housing['BsmtFullBath'] = housing['BsmtFullBath'].astype('object')
housing['BsmtHalfBath'] = housing['BsmtHalfBath'].astype('object')
housing['FullBath'] = housing['FullBath'].astype('object')
housing['HalfBath'] = housing['HalfBath'].astype('object')
housing['BedroomAbvGr'] = housing['BedroomAbvGr'].astype('object')
housing['KitchenAbvGr'] = housing['KitchenAbvGr'].astype('object')
housing['TotRmsAbvGrd'] = housing['TotRmsAbvGrd'].astype('object')
housing['Fireplaces'] = housing['Fireplaces'].astype('object')
housing['GarageCars'] = housing['GarageCars'].astype('object')

In [ ]:
housing.shape

#### Now we will check for any NAN values if any, if the count is less, we will remove them directly.

In [ ]:
housing.isnull().values.any()

In [ ]:
housing.isnull().sum().sum()

In [ ]:
housing.dropna(inplace=True)

In [ ]:
housing.shape

### Mapping of variables

#### We will map the necessary categorical variables 

In [ ]:
# List of variables to map

varlist1 =  ['Street']

# Defining the map function
def binary_map(x):
    return x.map({'Pave': 1, "Grvl": 0})

# Applying the function to the Lead list
housing[varlist1] = housing[varlist1].apply(binary_map)

In [ ]:
#CentralAir

varlist2 =  ['CentralAir']

# Defining the map function
def binary_map(x):
    return x.map({'Y': 1, "N": 0})

# Applying the function to the Lead list
housing[varlist2] = housing[varlist2].apply(binary_map)

In [ ]:
#Utilities

varlist3 =  ['Utilities']

# Defining the map function
def binary_map(x):
    return x.map({'AllPub': 1, "NoSeWa": 0})

# Applying the function to the Lead list
housing[varlist3] = housing[varlist3].apply(binary_map)

In [ ]:
housing['Utilities']

#### We will dropping Id column as it is of no use in our analysis

In [ ]:
drop_columns= ['Id']
housing= housing.drop(drop_columns,axis=1)

In [ ]:
np.isnan(housing.any())


### Data Preparation

In [ ]:
# creating dummy variables for categorical variables

# subset all categorical variables
house_categorical = housing.select_dtypes(include=['object'])
house_categorical.head()

In [ ]:
# convert into dummies
house_dummies = pd.get_dummies(house_categorical, drop_first=True)
house_dummies.head()

In [ ]:
# drop categorical variables 
housing = housing.drop(list(house_categorical.columns), axis=1)

In [ ]:
housing.head()

In [ ]:
# concat dummy variables with X
housing = pd.concat([housing, house_dummies], axis=1)

In [ ]:
housing.head()


## Splitting the Data into Training and Testing Sets

In [ ]:
from sklearn.model_selection import train_test_split

# We specify this so that the train and test data set always have the same rows, respectively
np.random.seed(0)
df_train, df_test = train_test_split(housing, train_size = 0.7, test_size = 0.3, random_state = 100)

##### Rescaling the Features 

We will use MinMax scaling.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
# Apply scaler() to all the columns except the 'yes-no' and 'dummy' variables
num_vars = ['LotFrontage','YearBuilt','YearRemodAdd','LotArea', 'TotalBsmtSF', 'GrLivArea',
           'MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GarageYrBlt',
            'GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MoSold','YrSold','SalePrice'
           ]

df_train[num_vars] = scaler.fit_transform(df_train[num_vars])

df_train.head()

In [ ]:
df_train.tail()

##### Dividing into X and Y sets for the model building

In [ ]:
y_train = df_train.pop('SalePrice')
X_train = df_train

In [ ]:
# Test Split-
y_test = df_test.pop('SalePrice')
X_test = df_test

In [ ]:
X_train.isnull().values.any()

### Applying RFE in order to reduce the count to 30 most important predictor variables.

In [ ]:
# Importing RFE and LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [ ]:
# Running RFE with the output number of the variable equal to 50
lm = LinearRegression()
lm.fit(X_train, y_train)

rfe = RFE(lm, 30)             # running RFE
rfe = rfe.fit(X_train, y_train)

In [ ]:
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

In [ ]:
#RFE Support columns
col = X_train.columns[rfe.support_]
col

In [ ]:
#Not RFE support columns

X_train.columns[~rfe.support_]

## Applying Ridge and Lasso

### Ridge

In [ ]:
# list of alphas to tune
params = {'alpha': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5,1.0, 5.0, 10.0, 100]}


ridge = Ridge()

# cross validation
folds = 5
model_cv = GridSearchCV(estimator = ridge, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)            
model_cv.fit(X_train, y_train) 

In [ ]:
#checking the value of optimum number of parameters
print(model_cv.best_params_)
print(model_cv.best_score_)

#### We got alpha value as 5.0 for Ridge

In [ ]:
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results = cv_results[cv_results['param_alpha']<=200]
cv_results.head()

In [ ]:
# plotting mean test and train scoes with alpha 
cv_results['param_alpha'] = cv_results['param_alpha'].astype('int32')

# plotting
plt.plot(cv_results['param_alpha'], cv_results['mean_train_score'])
plt.plot(cv_results['param_alpha'], cv_results['mean_test_score'])
plt.xlabel('alpha')
plt.ylabel('Negative Mean Absolute Error')
plt.title("Negative Mean Absolute Error and alpha")
plt.legend(['train score', 'test score'], loc='upper left')
plt.show()

In [ ]:
alpha = 5.0
ridge = Ridge(alpha=alpha)

ridge.fit(X_train, y_train)
ridge.coef_

In [ ]:
#lets predict the R-squared value of test and train data
y_train_pred = ridge.predict(X_train)
print('RSquare- '+str(metrics.r2_score(y_true=y_train, y_pred=y_train_pred)))

### LASSO

In [ ]:
lasso = Lasso()

# cross validation
model_cv = GridSearchCV(estimator = lasso, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)            

model_cv.fit(X_train, y_train) 

In [ ]:
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results.head()

In [ ]:
# plotting mean test and train scoes with alpha 
cv_results['param_alpha'] = cv_results['param_alpha'].astype('float32')

# plotting
plt.plot(cv_results['param_alpha'], cv_results['mean_train_score'])
plt.plot(cv_results['param_alpha'], cv_results['mean_test_score'])
plt.xlabel('alpha')
plt.xscale('log')
plt.ylabel('Negative Mean Absolute Error')

plt.title("Negative Mean Absolute Error and alpha")
plt.legend(['train score', 'test score'], loc='upper left')
plt.show()

In [ ]:
#checking the value of optimum number of parameters
print(model_cv.best_params_)
print(model_cv.best_score_)

In [ ]:
alpha = 0.001

lasso = Lasso(alpha=alpha)
        
lasso.fit(X_train, y_train)

In [ ]:
#lets predict the R-squared value of test and train data
y_train_pred = lasso.predict(X_train)
print('RSquare- ' +str(metrics.r2_score(y_true=y_train, y_pred=y_train_pred)))

In [ ]:
alpha = 0.001

lasso = Lasso(alpha=alpha)
        
lasso.fit(X_train, y_train) 

In [ ]:
results= pd.DataFrame.from_dict(model_cv.cv_results_)

In [ ]:
results= results.sort_values(['param_alpha'])
alphas= np.array(params['alpha'])

In [ ]:
results[['param_alpha','mean_train_score','mean_test_score']]